In [3]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import os
import hashlib
import pandas as pd
from colorama import Fore, Style, init

init(autoreset=True)

URL = "https://www.valencia.es/cas/movilidad/otras-descargas"
CURRENT_YEAR = datetime.now().year
CURRENT_MONTH = datetime.now().strftime("%B").upper()

directorio_base = os.getcwd()
while not os.path.exists(os.path.join(directorio_base, "Datos")) and directorio_base != "/":
    directorio_base = os.path.dirname(directorio_base)

directorio_descargas = os.path.join(directorio_base, "Datos", "IMDS_Trafico")
os.makedirs(directorio_descargas, exist_ok=True)

ruta_hash = os.path.join(directorio_descargas, "hash_imds.txt")

def hash_dataframe(df):
    csv_bytes = df.to_csv(index=False).encode("utf-8")
    return hashlib.md5(csv_bytes).hexdigest()

def clean_imd_column(df):
    imd_col = [col for col in df.columns if "IMD" in col.upper()][0]
    df[imd_col] = df[imd_col].astype(str).str.replace('.', '', regex=False).astype(int)
    return df

def get_download_link(html, year):
    soup = BeautifulSoup(html, "html.parser")
    panels = soup.find_all("div", class_="panel panel-default")

    for panel in panels:
        heading = panel.find("div", class_="panel-heading")
        if heading and str(year) in heading.text:
            body = panel.find("div", class_="panel-body")
            if body:
                links = body.find_all("a", href=True)
                for link in links:
                    href = link['href']
                    text = link.get_text(strip=True)
                    if "Imds vehículos motorizados" in text and href.endswith(".ods"):
                        return href
    return None

def archivo_actualizado(df_nuevo):
    nuevo_hash = hash_dataframe(df_nuevo)

    if not os.path.exists(ruta_hash):
        return True, nuevo_hash

    with open(ruta_hash, "r") as f:
        hash_anterior = f.read().strip()

    return nuevo_hash != hash_anterior, nuevo_hash

def main():
    print(f"{Fore.YELLOW}🔍 Buscando archivo en la web...")
    response = requests.get(URL)
    response.raise_for_status()

    ods_link = get_download_link(response.text, CURRENT_YEAR)
    if not ods_link:
        print(f"{Fore.RED}❌ No se encontró el archivo del año actual.")
        return

    print(f"{Fore.BLUE}📥 Descargando archivo: {Style.BRIGHT}{ods_link}")
    file_data = requests.get(ods_link).content

    temp_path = os.path.join(directorio_descargas, "temp.ods")
    with open(temp_path, "wb") as f:
        f.write(file_data)

    try:
        xls = pd.read_excel(temp_path, sheet_name=None, engine="odf")
    except Exception as e:
        print(f"{Fore.RED}⚠️ Error leyendo el archivo: {e}")
        os.remove(temp_path)
        return

    possible_sheets = [k for k in xls.keys() if CURRENT_MONTH[:3] in k.upper()]
    if not possible_sheets:
        print(f"{Fore.RED}❌ No se encontró hoja para el mes actual ({CURRENT_MONTH}).")
        os.remove(temp_path)
        return

    sheet_name = possible_sheets[0]
    df_new = clean_imd_column(xls[sheet_name])
    actualizado, nuevo_hash = archivo_actualizado(df_new)

    if not actualizado:
        print(f"{Fore.YELLOW}🔄 El archivo no ha cambiado. Se elimina el temporal.")
        os.remove(temp_path)
        return

    fecha_str = datetime.now().strftime("%d_%m_%y")
    nombre_csv = f"imds_{fecha_str}.csv"
    ruta_csv = os.path.join(directorio_descargas, nombre_csv)

    df_new.to_csv(ruta_csv, index=False)
    with open(ruta_hash, "w") as f:
        f.write(nuevo_hash)

    os.remove(temp_path)
    print(f"{Fore.GREEN}✅ Nuevo archivo guardado en: {Style.BRIGHT}{ruta_csv}")

if __name__ == "__main__":
    main()

2025
🔍 Buscando archivo en la web...
❌ No se encontró el archivo del año actual.
